# Predicting California’s Groundwater Levels Using Machine Learning for Sustainable Management

In [2]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

## Fetching the Groundwater Level Measurements via the CA Dept. of Water Resources API

In [3]:
# Fetch data from API
def fetch_groundwater_data():
    sql_query = "SELECT * FROM \"231ba173-392a-4f62-91fb-07f4a90746d1\" WHERE county_name = 'Fresno'"
    url = f"https://data.ca.gov/api/3/action/datastore_search_sql?sql={sql_query}"

    response = requests.get(url)
    
    # Check if request was successful
    if response.status_code == 200:
        data = response.json()
        records = data['result']['records']
        df = pd.DataFrame(records)
        
        # Print basic details
        print("\nData Successfully Fetched!")
        print(f"Total Records Retrieved: {len(df)}")
        print("\nPreview of Data:\n", df.head())  # Print first few rows
        
        return df
    else:
        print("Failed to fetch data. Status Code:", response.status_code)
        return None

# Call function and print results
df = fetch_groundwater_data()

# Optional: Display full column names to understand the dataset better
if df is not None:
    print("\nColumns in DataFrame:\n", df.columns.tolist())


print(f"Total Number of Rows in Dataset: {df.shape[0]}")



Data Successfully Fetched!
Total Records Retrieved: 50000

Preview of Data:
        _id _full_text           site_code            msmt_date  wlm_rpe  \
0  1413344       None  359714N1201027W001  2023-04-20T07:44:00  697.120   
1  1383070       None  357878N1191173W001  1977-12-01T00:00:00  508.200   
2  1383071       None  357878N1191173W001  1977-01-03T00:00:00  508.200   
3  1383072       None  357878N1191173W001  1976-01-08T00:00:00  508.200   
4  1383073       None  357878N1191173W001  1974-03-06T00:00:00  508.200   

   wlm_gse      gwe  gse_gwe   wlm_qa_desc             wlm_qa_detail  \
0  695.120     None     None       Missing  Temporarily inaccessible   
1  508.200  202.200  306.000          Good                      None   
2  508.200  240.700  267.500          Good                      None   
3  508.200  271.000  237.200  Questionable                     Other   
4  508.200  227.200  281.000          Good                      None   

  wlm_mthd_desc wlm_acc_desc          

## Fetch and Focus on Fresno County data for better accuracy and less variability

In [4]:
# Call function and get DataFrame
df_fresno = fetch_groundwater_data()

# Save to CSV (optional)
if df_fresno is not None:
    save_csv = True  # Set to False if you don't want to save
    if save_csv:
        df_fresno.to_csv("fresno_groundwater.csv", index=False)
        print("\Fresno County data saved as 'fresno_groundwater.csv'")


Data Successfully Fetched!
Total Records Retrieved: 50000

Preview of Data:
        _id _full_text           site_code            msmt_date  wlm_rpe  \
0  1423046       None  360219N1199616W001  1972-01-04T00:00:00  208.070   
1  1383070       None  357878N1191173W001  1977-12-01T00:00:00  508.200   
2  1383071       None  357878N1191173W001  1977-01-03T00:00:00  508.200   
3  1383072       None  357878N1191173W001  1976-01-08T00:00:00  508.200   
4  1383073       None  357878N1191173W001  1974-03-06T00:00:00  508.200   

   wlm_gse      gwe  gse_gwe   wlm_qa_desc          wlm_qa_detail  \
0  208.070     None     None       Missing  Unable to locate well   
1  508.200  202.200  306.000          Good                   None   
2  508.200  240.700  267.500          Good                   None   
3  508.200  271.000  237.200  Questionable                  Other   
4  508.200  227.200  281.000          Good                   None   

  wlm_mthd_desc wlm_acc_desc                   wlm_org_n

## Missing Values

In [5]:
# Data Preprocessing
def preprocess_data(df):
    required_columns = ['measurement_date', 'site_code', 'gs_elev', 'rp_elev', 'wl_below_surface']
    available_columns = df.columns.tolist()
    missing_columns = [col for col in required_columns if col not in available_columns]
    
    if missing_columns:
        print(f"Missing columns: {missing_columns}")
        return pd.DataFrame()
    
    df = df[required_columns]
    df.dropna(inplace=True)
    df['measurement_date'] = pd.to_datetime(df['measurement_date'], errors='coerce')
    df.dropna(subset=['measurement_date'], inplace=True)
    df.sort_values(by=['measurement_date'], inplace=True)
    
    label_encoder = LabelEncoder()
    df['site_code'] = label_encoder.fit_transform(df['site_code'])
    return df


In [6]:
# Feature Engineering
def feature_engineering(df):
    df['year'] = df['measurement_date'].dt.year
    df['month'] = df['measurement_date'].dt.month
    df['day'] = df['measurement_date'].dt.day
    df.drop(columns=['measurement_date'], inplace=True)
    return df

raw_df = fetch_groundwater_data()
if not raw_df.empty:
    processed_df = preprocess_data(raw_df)
    if not processed_df.empty:
        processed_df = feature_engineering(processed_df)
        processed_df.to_csv("groundwater_data.csv", index=False)
    else:
        print("Data preprocessing failed. No further steps executed.")
else:
    print("Data fetching failed. No further steps executed.")


Data Successfully Fetched!
Total Records Retrieved: 50000

Preview of Data:
        _id _full_text           site_code            msmt_date  wlm_rpe  \
0  1383070       None  357878N1191173W001  1977-12-01T00:00:00  508.200   
1  1383071       None  357878N1191173W001  1977-01-03T00:00:00  508.200   
2  1383072       None  357878N1191173W001  1976-01-08T00:00:00  508.200   
3  1383073       None  357878N1191173W001  1974-03-06T00:00:00  508.200   
4  1383074       None  357878N1191173W001  1972-02-07T00:00:00  508.200   

   wlm_gse      gwe  gse_gwe   wlm_qa_desc wlm_qa_detail wlm_mthd_desc  \
0  508.200  202.200  306.000          Good          None       Unknown   
1  508.200  240.700  267.500          Good          None       Unknown   
2  508.200  271.000  237.200  Questionable         Other       Unknown   
3  508.200  227.200  281.000          Good          None       Unknown   
4  508.200  216.500  291.700          Good          None       Unknown   

  wlm_acc_desc            

In [7]:
# Splitting Data
X = processed_df.drop(columns=['wl_below_surface'])
y = processed_df['wl_below_surface']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


KeyError: "['wl_below_surface'] not found in axis"